# This is PMS data analysis of the videos

Made in environment **PMS_Study**

TO DO: bekijk file PMS_analysis/2_preparation

Probeer veel met loops en functions te werken om de code overzichtelijk te houden

 * Start with file cleanData.csv (geeft informatie over van welke participant welke files bestaan)
 * Load timings.csv file that Tilia and Sofie made
 * Start loop from the top and use Order and A-B
 * In loop check if file (corresponding CSV) exists in Directory (/10001 or /10002 depedent of A/B): later refered to as behaviordata
 * In loop check if file (corresponding .WEBM) exists in Directory
 
If both exist:
- Check if output of Openface exists in Z:\shares\ghepmk_data\2020_Kappen_PMS\10002\video_features (or /10001 dependent of A/B)
- IF True:
Load this as facedata
- IF False:
Print: "Video file present for file (insert filename) but no openface present)"
Break/skip

Use filename/pptnum to search for right starting time in timings.csv variable
- Find timing for beep.wav in behaviordata. 
- Set new zero point (link between the timings.csv point and this)
- Make mean scores on a few variables (we will expand this later) for the corresponding frames of each of the trials.
- To do this, you should write a function that takes the right points of a trial (start, end, trialnum etc) and save this data to a dataframe

Compute this in a way that you would be able to add it to allPMSdata.cvs

Add it all to a new dataframe

Write to csv

UsageError: Line magic function `%magiclb` not found.


In [1]:
%matplotlib inline

In [24]:
import glob
import os
import subprocess
import numpy as np
import pandas as pd
import math

from tqdm import tqdm
from pathlib import Path

import re



In [25]:
# Declare all directories:

data_dir = Path('Z:/shares/ghepmk_data/2020_Kappen_PMS')

# Create all relevant directories
date_dir = "24082021/"
data_all_dir = os.path.join(data_dir, date_dir)
data_A_dir = os.path.join(data_dir, Path('10001'))
data_B_dir = os.path.join(data_dir, Path('10002'))



## uploading the Data
### Testmoment 1 and 2

In [34]:

df = pd.read_csv("Z:/shares/ghepmk_data/2020_Kappen_PMS/24082021/cleanData.csv", encoding = "ISO-8859-1")

A = pd.read_csv("Z:/shares/ghepmk_data/2020_Kappen_PMS/10001/10001.csv")
A.head()


ParserError: Error tokenizing data. C error: Expected 3 fields in line 9, saw 4
